In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
df= pd.read_csv("data.csv")
df.dropna(inplace=True)
df.head()

,Equipment SNO,Asset Number,Reading Name,Answer Value,Timestamp,Is Error Set?,Asset,Company
0,68B6B34180C8-3,FSCHN-E-00001,activeenergydla,76646.43,20/02/24 15:58,False,AHU DB,Chennai – Bayline
1,68B6B34180C8-3,FSCHN-E-00001,activeenergydla,76641.62,20/02/24 15:51,False,AHU DB,Chennai – Bayline
2,68B6B34180C8-3,FSCHN-E-00001,activeenergydla,76636.78,20/02/24 15:44,False,AHU DB,Chennai – Bayline
3,68B6B34180C8-3,FSCHN-E-00001,activeenergydla,76633.33,20/02/24 15:39,False,AHU DB,Chennai – Bayline
4,68B6B34180C8-3,FSCHN-E-00001,activeenergydla,76628.46,20/02/24 15:32,False,AHU DB,Chennai – Bayline


In [2]:


import pandas as pd
from prophet import Prophet# 

df = df.drop_duplicates()
missing_values = df.isnull().sum()
print("Missing values:\n", missing_values)


df['Timestamp'] = pd.to_datetime(df['Timestamp'])


df = df.sort_values('Timestamp')

Missing values:
 Equipment SNO    0
Asset Number     0
Reading Name     0
Answer Value     0
Timestamp        0
Is Error Set?    0
Asset            0
Company          0
dtype: int64


/var/folders/bp/6nr_cvms0_x4_qdd1ntdd17c0000gn/T/ipykernel_96909/3799681826.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Timestamp'] = pd.to_datetime(df['Timestamp'])


In [3]:
df['Hour'] = df['Timestamp'].dt.hour
df['DayOfWeek'] = df['Timestamp'].dt.dayofweek
df['DayOfMonth'] = df['Timestamp'].dt.day
df['Month'] = df['Timestamp'].dt.month
df['Year'] = df['Timestamp'].dt.year
scaler = MinMaxScaler()
numerical_cols = ['Answer Value', 'Hour', 'DayOfWeek', 'DayOfMonth', 'Month', 'Year']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [4]:
import pandas as pd
from prophet import Prophet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values('Timestamp')
df = df.rename(columns={'Timestamp': 'ds', 'Answer Value': 'y'})
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=False)
prophet_model = Prophet()
prophet_model.fit(train_data)
future = prophet_model.make_future_dataframe(periods=len(test_data))
forecast = prophet_model.predict(future)
actual_values = test_data['y'].values
predicted_values = forecast.iloc[-len(test_data):]['yhat'].values
mae = mean_absolute_error(actual_values, predicted_values)
mse = mean_squared_error(actual_values, predicted_values)
rmse = np.sqrt(mse)
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)


16:21:45 - cmdstanpy - INFO - Chain [1] start processing


In [ ]:

future_dates = prophet_model.make_future_dataframe(periods=365)  
forecast = prophet_model.predict(future_dates)

forecast_values = forecast[['ds', 'yhat']]

print(forecast_values.tail())  


                      ds      yhat
9613 2025-06-27 09:23:00 -0.030264
9614 2025-06-28 09:23:00  0.018044
9615 2025-06-29 09:23:00  0.088046
9616 2025-06-30 09:23:00 -0.061462
9617 2025-07-01 09:23:00  0.116502


In [ ]:

future_dates_2months = prophet_model.make_future_dataframe(periods=60)  


forecast_2months = prophet_model.predict(future_dates_2months)


forecast_values_2months = forecast_2months[['ds', 'yhat']]


print(forecast_values_2months.tail()) 


                      ds      yhat
9308 2024-08-26 09:23:00  0.178110
9309 2024-08-27 09:23:00  0.356075
9310 2024-08-28 09:23:00  0.244022
9311 2024-08-29 09:23:00  0.190222
9312 2024-08-30 09:23:00  0.203864


In [ ]:
# Generate future timestamps for forecasting two months before 01-01-2024
future_dates_before = pd.date_range(start='2023-11-01', end='2023-12-31')

# Make predictions for the future timestamps
forecast_before = prophet_model.predict(pd.DataFrame({'ds': future_dates_before}))

# Extract forecasted values for two months before 01-01-2024
forecast_values_before = forecast_before[['ds', 'yhat']]

# Display the forecasted values for two months before 01-01-2024
print("Forecast for two months before 01-01-2024:")
print(forecast_values_before.tail())  # Display the last few forecasted values for two months before 01-01-2024

# Generate future timestamps for forecasting two months after 20-02-2024
future_dates_after = pd.date_range(start='2024-04-20', periods=60)

# Make predictions for the future timestamps
forecast_after = prophet_model.predict(pd.DataFrame({'ds': future_dates_after}))

# Extract forecasted values for two months after 20-02-2024
forecast_values_after = forecast_after[['ds', 'yhat']]

# Display the forecasted values for two months after 20-02-2024
print("\nForecast for two months after 20-02-2024:")
print(forecast_values_after.head())  # Display the first few forecasted values for two months after 20-02-2024


Forecast for two months before 01-01-2024:
           ds      yhat
56 2023-12-27  0.311171
57 2023-12-28  0.151807
58 2023-12-29  0.205257
59 2023-12-30  0.182094
60 2023-12-31  0.327468

Forecast for two months after 20-02-2024:
          ds      yhat
0 2024-04-20  0.299171
1 2024-04-21  0.443341
2 2024-04-22  0.304271
3 2024-04-23  0.375446
4 2024-04-24  0.426415


In [ ]:
import plotly.graph_objects as go

trace_actual = go.Scatter(x=df['ds'], y=df['y'], mode='markers', name='Actual',
                          marker=dict(color='blue', symbol='circle', size=8),
                          line=dict(color='blue', width=2))

trace_forecast = go.Scatter(x=forecast_combined['ds'], y=forecast_combined['yhat'], mode='markers', name='Forecast',
                            marker=dict(color='red', symbol='x', size=10),
                            line=dict(color='red', width=2))

layout = go.Layout(title='Actual vs Forecasted Energy Consumption',
                   xaxis=dict(title='Timestamp',
                              tickmode='array',
                              tickvals=df['ds'],  # Set the tick values to the actual dates
                              ticktext=df['ds'].dt.strftime('%Y-%m-%d %H:%M')),  # Format the tick labels as desired
                   yaxis=dict(title='Energy Consumption'))

fig = go.Figure(data=[trace_actual, trace_forecast], layout=layout)

fig.show()
